In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Seguindo o guia "EDA DieTanic", a linha de comando 'import warnings', foi pesquisado que esta biblioteca serve para emitir alertas durante o processo de importar módulos ( ignorado por default).Também a linha de comando "%matplotlib inline" serve para exibir plots em notebooks.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
print("ok")

In [ ]:
#Vamos ler as 3 bases de dados
calendar=pd.read_csv('../input/airbnb-rome/calendar.csv')
listings=pd.read_csv('../input/airbnb-rome/listings.csv')
reviews=pd.read_csv('../input/airbnb-rome/reviews.csv')
print('ok')

In [ ]:
calendar.head()

In [ ]:
calendar.describe()

In [ ]:
listings.head()

In [ ]:
reviews.head()

In [ ]:
print(listings.columns)
#listando as colunas do dataframe 'listings'

**De início, pensei em juntar as dataframes "listings" + "reviews" + "calendar" em formato outer para não perder nenhuma informação. Porém ,verifiquei que a "reviews" é supérflua. Além disso,a dataframe "calendar" tem já informação redundante de'listings'(ver lista abaixo):
('price','minimum_nights','maximum_nights','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights,'minimum_nights_avg_ntm''maximum_nights_avg_ntm','calendar_updated','has_availability','availability_30,'availability_60','availability_90','availability_365','calendar_last_scraped').
Portanto, após olhar a coluna "calendar_updated" de listings, só irei trabalhar com ela.**

In [ ]:
listings["calendar_updated"]

In [ ]:
listings["calendar_updated"].describe()

**A coluna "calendar_updated" não tem valores. Se eu quiser fazer alguma avaliação temporal, terei de empregar o dataframe "calendar**"

In [ ]:
listings['calendar_last_scraped'].describe()

In [ ]:
from pprint import pprint
pprint(listings.dtypes)
print('ok')

In [ ]:
print(listings.price.dtypes)
print(listings.minimum_nights.dtypes)

In [ ]:
#listings.price = pd.to_numeric(listings.price, downcast="float")
#listings.price = listings.price.astype('float64')
#converter coluna 'price' para float type
for a in range(0,len(listings.price)):
    listings.price[a] = listings.price[a].replace("$","").replace(",","") 
    listings.price[a] = float(listings.price[a])
print(listings.price.dtypes)
listings['price'].describe()

In [ ]:
listings.price.tail()
#tentando descobrir pq a última posição é string "$36.00"

In [ ]:
listings.isnull().sum()
pprint(listings.isnull().sum())

In [ ]:
listings.groupby(['price','minimum_nights'])['minimum_nights'].count()

In [ ]:
from bokeh.plotting import figure,show

from bokeh.io import output_file,show,output_notebook,push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,CategoricalColorMapper
from bokeh.layouts import row,column,gridplot
from bokeh.models.widgets import Tabs,Panel
output_notebook()
# fazendo um graph
p = figure(plot_width=500, plot_height=500)
p.xaxis.axis_label = 'Price $'
p.yaxis.axis_label = 'Minimum Nights'
# add a circle renderer with a size, color, and alpha
p.circle(x=listings['price'],y=listings['minimum_nights'],size=2, color="navy", alpha=0.5)
# show the results
show(p)

In [ ]:
p = figure(title="Minimum Nights vs Price", x_axis_label="Price($)", y_axis_label="Minimum Nights")
p.vbar(x=listings['price'], top=listings['minimum_nights'], width=2, bottom=0, color="red")
show(p)

In [ ]:
listings.groupby('neighbourhood').price.min()

In [ ]:
listings.groupby('neighbourhood').price.max()

In [ ]:
listings.price.mean()

In [ ]:
listings.price.median()

In [ ]:
listings.price.quantile(q=0.25)

In [ ]:
listings.price.quantile(q=0.75)

In [ ]:
listings.price.mode()

In [ ]:
listings.price.max()

In [ ]:
listings.price.min()

In [ ]:
listings.groupby(['latitude','longitude']).price.min()

In [ ]:
listings.price.std()

In [ ]:
listings.groupby(['latitude','longitude']).price.max()

In [ ]:
highprice_loc = listings.groupby(['latitude','longitude']).price.max().sort_values(ascending=False)

In [ ]:
listings.groupby(['latitude','longitude']).price.max().sort_values(ascending=False)

In [ ]:
import folium
m = folium.Map(location=[41.902782, 12.51133], zoom_start=12, tiles="OpenStreetMap")
tooltip = "Click me!"
#highprice_loc.index

for a in range(0,10):
    text = str(highprice_loc.iloc[a])
    folium.Marker([highprice_loc.index[a][0], highprice_loc.index[a][1]], popup=f"<i>{text}</i>", tooltip=tooltip,icon=folium.Icon(color='lightgray', icon='home', prefix='fa')).add_to(m)
    
for a in range(25910,25920):
    text = str(highprice_loc.iloc[a])
    folium.Marker([highprice_loc.index[a][0], highprice_loc.index[a][1]],popup=f"<i>{text}</i>", tooltip=tooltip,icon=folium.Icon(color="red", icon="info-sign")).add_to(m)
        
m

**Acima podemos observar as coordenadas geográficas agrupadas pelo preço máximo com os 10 mais caros( cinza) e os 10 mais baratos (vermelho). Os nove mais caros se concentram próximo da Ponte Cavour e da Praça Imperador Augusto. Os 10 mais baratos locais custam $0(zero), $8 e $9 .**

In [ ]:
listings.groupby(['latitude','longitude']).price.min().sort_values(ascending=False)

**Abaixo iremos exibir no mapa os locais com pontuação mais alta e mais baixa sobre limpeza. Embora estes locais estejam relativamente espalhados, observar que os locais mais limpos tendem a se concentrar perto do Vaticano.**

In [ ]:
import folium
m = folium.Map(location=[41.902782, 12.51133], zoom_start=12, tiles="OpenStreetMap")
tooltip = "Click me!"

cleaniness = listings.groupby(['latitude','longitude']).review_scores_cleanliness.max().sort_values(ascending=False)
for a in range(0,10):
    text = str(cleaniness.iloc[a])
    folium.Marker([cleaniness.index[a][0], cleaniness.index[a][1]], popup=f"<i> Ponto de Limpeza {text}</i>", tooltip=tooltip,icon=folium.Icon(color='lightgray', icon='home', prefix='fa')).add_to(m)

dirtiness = listings.groupby(['latitude','longitude']).review_scores_cleanliness.max().sort_values(ascending=True)
for a in range(0,10):
    text = str(dirtiness.iloc[a])
    folium.Marker([dirtiness.index[a][0], dirtiness.index[a][1]],popup=f"<i>Ponto de Sujeira {text}</i>", tooltip=tooltip,icon=folium.Icon(color="red", icon="info-sign")).add_to(m)
        
m

**Abaixo iremos fazer "binning" da coluna "price" para facilitar nossas análises.**

In [ ]:
listings['quartil_price'] = pd.cut(listings['price'], bins=4, labels=['1 quartil $', '2 quartil $', '3 quartil $', '4 quartil $'])
print('ok')

In [ ]:
listings['quartil_price'].head()

In [ ]:
listings['quartil_price'].tail()

In [ ]:
 listings.groupby(['quartil_price']).minimum_nights.mean().sort_values(ascending=False)

In [ ]:
 listings.groupby(['quartil_price']).minimum_nights.median().sort_values(ascending=False)

In [ ]:
#pd.crosstab(listings.price,listings['minimum_nights'],margins=True).style.background_gradient(cmap='summer_r')
#a crosstab é de difícil paginação
sns.factorplot('minimum_nights','neighbourhood', hue='quartil_price',data=listings)
fig = plt.gcf()
fig.set_size_inches(20.5, 20.5, forward=True)
plt.show()
"factorplot não gerou bons resultados para interpretação. Podemos ver que o local 'Rome,Lazio,Italy' é" 
"onde se concentra os 3 maiores quartis de preço."

In [ ]:
listings['price'].dtypes

In [ ]:
listings['price'].describe()

In [ ]:
listings['minimum_nights'].describe()

In [ ]:
listings['minimum_nights'].dtypes

In [ ]:
listings['minimum_nights'].astype('float64').corr(listings['price'].astype('float64'))

In [ ]:
lista =['accommodates', 'bathrooms','bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price','minimum_nights', 'maximum_nights']
for i in lista:
    print(listings[i].dtypes)

In [ ]:
listings['bathrooms_text'].describe()

In [ ]:
listings['amenities'].describe()

In [ ]:
listings['bathrooms'].describe()

In [ ]:
lista_num =['accommodates', 'bathrooms', 'bedrooms', 'beds','price','minimum_nights', 'maximum_nights']
for i in lista_num:
    a = listings[i].astype('float64').corr(listings['price'].astype('float64'))
    txt = f" A correlação entre {i} e 'price' é: {str(a)}"
    print(txt)

**Embora nos 2 gráficos acima ( o de barras e o tipo scatterplot) visualmente se verifique que há relação inversa entre a quantidade de noites mínimas e o preço da acomodação, estatisticamente não se verifica isso, visto que a Correlação de Pearson para ambos é -0.009.**

In [ ]:
#df100 = listings.loc[(listings['price'] <= 100)]
pd.crosstab(listings.quartil_price,listings['bedrooms'],margins=True).style.background_gradient(cmap='summer_r')

**Observa-se o elevado número de acomodações de 1 quarto cujo preço de acomodação está no quartil inferior.**

In [ ]:
#listings['price'].describe()
#listings['price'].mean()
cross = pd.crosstab(index=listings['quartil_price'],columns=listings['neighbourhood'],values=listings['bedrooms'],aggfunc=np.mean)
plt.figure(figsize=(20, 20))
sns.heatmap(cross, cmap='rocket_r',annot=True, fmt='g')

**Podemos observar que no Bairro Lidio de Ostia, há uma média de 1,5 quartos no quartil mais caro de preço. Já no Bairro Formello há a maior média de número de quartos (8) no quartil mais barato.**

In [ ]:
listings['review_scores_location'].describe()

**Agora iremos fazer um mapa de calor com a média de "review_scores_location"(pontuação sobre o local de instalação) vs ( quartil de preço & bairro). Observem que as avaliações para os 3 quartis superiores está em torno de 5 pontos.** 

In [ ]:
cross = pd.crosstab(index=listings['quartil_price'],columns=listings['neighbourhood'],values=listings['review_scores_location'],aggfunc=np.mean)
plt.figure(figsize=(20, 20))
sns.heatmap(cross, cmap='rocket_r',annot=True, fmt='g')

In [ ]:
listings['accommodates'].describe()

**Abaixo iremos representar um gráfico de violino onde são confrontados Camas vs Quartos por Disponibilidade**

In [ ]:
plt.figure(figsize=(20, 20))
sns.violinplot(x="beds", y="bedrooms", hue="has_availability",data=listings,split=True)


**Como podemos ver no gráfico acima, a mediana de quartos aumenta à medida que o número de camas aumenta.Podemos observar também que que quartos que tenham de 1 até 6 camas podem se encontrar indisponíveis.Podemos notar também a enorme variância de quantidade de quartos que possuem de 1 até 2 camas.Pelo formato da curva, pode-se pensar em um possível outlier, pois a densidade de pontos nos extremos superiores é quase nula.**

In [ ]:
calendar['price'] = calendar['price'].str.slice(start=1)
#removi o $ da coluna price

In [ ]:
calendar['price'] = calendar['price'].str.slice(start=0,stop=-3)

In [ ]:
calendar['price'] = calendar['price'].str.replace("[^0-9]", "", n=-1, case=None, regex=True)

In [ ]:
calendar['price'][6445]

In [ ]:
calendar['price'].head()

In [ ]:
calendar['price'] = pd.to_numeric(calendar['price'], errors='raise')

In [ ]:
calendar['date'].describe()

In [ ]:
calendar['price'].describe()


In [ ]:
calendar['price'].head()

In [ ]:
type(calendar['price'][0])

In [ ]:
calendar['price'].tail()

In [ ]:
 calendar.groupby(['date']).price.mean().sort_values(ascending=False)

In [ ]:
 calendar.groupby(['date']).price.max().sort_values(ascending=False)

In [ ]:
 calendar.groupby(['date']).price.min().sort_values(ascending=False)

In [ ]:
 calendar.groupby(['date']).price.median().sort_values(ascending=False)

In [ ]:
coluna_max = calendar.groupby(['date']).price.max()
coluna_media = calendar.groupby(['date']).price.mean()
coluna_mediana =calendar.groupby(['date']).price.median()
coluna_min = calendar.groupby(['date']).price.min()

In [ ]:
plt.figure(figsize=(10, 10))
coluna_max.plot.line(x='date')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 5))
coluna_max.plot(ax=axes[0,0]); axes[0,0].set_title('Preço Máximo vs Data')
coluna_media.plot(ax=axes[0,1]); axes[0,1].set_title('Preço Média vs Data')
coluna_mediana.plot(ax=axes[1,0]); axes[1,0].set_title('Preço Mediano vs Data')
coluna_min.plot(ax=axes[1,1]); axes[1,1].set_title('Preço Mínimo vs Data')

In [ ]:
plt.figure(figsize=(10, 10))
#coluna_max.plot(color='r')
coluna_media.plot(color='g')
coluna_mediana.plot(color='b')

In [ ]:
plt.figure(figsize=(10, 10))
coluna_max.plot(color='r')
coluna_min.plot(color='b')

In [ ]:
coluna_media.max()

In [ ]:
coluna_media.loc[coluna_media == 173.96012202293443]

In [ ]:
coluna_mediana.max()

In [ ]:
coluna_mediana.loc[coluna_mediana == 85.0]

****Como podemos ver acima, o dia 31/12/21 é o dia onde haverá o pico de preço de hospedagem, sendo o valor médio de 174 dólares e 85 dólares a mediana do valor. Os valores de hospedagem  mais caros têm um comportamento periodico tipo onda triangular, oscilando entre 10000 e 12000 dólares. ****

In [ ]:
calendar.describe()

**Observar que média de preços é 1.424167e+02, mas seu desvio avaliado é 4.013013e+02 e sua mediana é 7.900000e+01. Isso se deve ao seu valor máximo, de 1.200000e+04.**

In [ ]:
calendar.date.loc[calendar.price == 1.200000e+04]

In [ ]:
from datetime import datetime,date
date(year=int(calendar.date[6847598][0:4]), month=int(calendar.date[6847598][5:7]), day=int(calendar.date[6847598][8:10])).isoweekday()

In [ ]:
calendar['week_days'] = calendar.date.map(lambda p: date(year=int(p[0:4]),month=int(p[5:7]), day=int(p[8:10])).isoweekday())

In [ ]:
calendar['week_days'].loc[calendar.price == 1.200000e+04]

In [ ]:
calendar['week_days'].describe()

In [ ]:
calendar['week_days'].replace(1, "Segunda").replace(2, "Terça").replace(3, "Quarta").replace(4, "Quinta").replace(5, "Sexta").replace(6, "Sábado").replace(7, "Domingo")

In [ ]:
calendar['week_days_name'] = calendar['week_days'].replace(1, "Segunda").replace(2, "Terça").replace(3, "Quarta").replace(4, "Quinta").replace(5, "Sexta").replace(6, "Sábado").replace(7, "Domingo")

In [ ]:
calendar['week_days_name'].describe()

In [ ]:
calendar['week_days_name'].loc[calendar.price == 1.200000e+04]

In [ ]:
calendar['week_days_name'].loc[calendar.price == 1.200000e+04].describe()

In [ ]:
calendar['week_days_name'].loc[calendar.price == 1.200000e+04].hist()

**Os dias da semana onde o preço é mais caro ( 12 mil dólares) são Sexta e Sábado.**

In [ ]:
listings.minimum_nights.describe()

In [ ]:
calendar.minimum_nights.describe()

In [ ]:
listings.head()

In [ ]:
calendar.head()

**As colunas em comum entre o Dataframe 'listings' e 'calendar' são : id/listing_id, price, minimum_nights, maximum_nights, available.Além da diferença do número de linhas entre ambas está na escala de 1000 vezes ( 27.000 vx 10.000.000), não há garantia que os ids sejam realmente comuns entre ambos e não há portanto como parear/juntar os dois dataframes, visto que um (o dataframe listings) foi obtido em somente 2 dias (2021-07-08 e 2021-07-07, quando foram feitos os scrapes).**

In [ ]:
reviews.head()

In [ ]:
!pip install text2emotion
import text2emotion as te

In [ ]:
reviews['comments'].map(str)

In [ ]:
type(reviews.comments[1041659])

In [ ]:
te.get_emotion(reviews.comments[0])

In [ ]:
#[^a-zA-Z]
import re
pattern = r'[0-9]'
reviews['comments_fix'] = reviews['comments'].str.replace(pattern, "", n=-1, case=None, regex=True)

In [ ]:
reviews['comments_fix'].map(str)

In [ ]:
reviews['comments_fix'][489]

In [ ]:
reviews['comments_fix'] = reviews['comments_fix'].replace(np.nan, '', regex=True)
print("ok")

In [ ]:
words = pd.Series(' '.join(reviews['comments_fix']).split()).value_counts()
print('ok')

In [ ]:
words

In [ ]:
words.loc['the']

In [ ]:
words.loc['and']==None

In [ ]:
words.loc['and']!=None

**Tentei usar a biblioteca 'text2emotion', mas ela demorou muito para executare não dava resposta.
Abaixo um exemplo de rotina que tentei executar.**

    te.get_emotion(reviews.comments[0])
    def d(p):
    q = te.get_emotion(p)
    o = {"Happy":0 , "Angry":0, "Surprise":0, "Sad":0,"Fear":0}
    try:
        q = te.get_emotion(p)
        return q
    except AttributeError:
        o = {"Happy":0 , "Angry":0, "Surprise":0, "Sad":0,"Fear":0}
        return o

    reviews['key_emotions'] = reviews.comments.apply(lambda p: d(p))
    reviews["key_emotions"] = reviews.comments_fix.map(lambda p: te.get_emotion(p))

In [ ]:
Sad= dict()
Sadx=["sad","depressed","dejected","heavy","crushed",
      "disgusted","upset","sorrowful","mournful","weepy","frustrated","dismayed"]
print("Sad Feelings Word Count")
for a in Sadx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Sad[a] = o
    except KeyError:
        continue
print(Sad)

In [ ]:
Happy=dict()
Happyx=["amused","delighted","glad","pleased","charmed","grateful","optimistic","content",
        "joyful","enthusiastic","loving","marvelous","marvellous","happy"]
print("Happy Feelings Word Count")
for a in Happyx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Happy[a] = o
    except KeyError:
        continue
print(Happy)

In [ ]:
Hurt=dict()
Hurtx=["hurt","jealous","betrayed","isolated","shocked","downtrodden",
 "wounded","impaired","damaged","criticized","abused","punished","rejected","forgotten","insulted"]
print("Hurt Feelings Word Count")
for a in Hurtx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Hurt[a] = o
    except KeyError:
        continue
print(Hurt)

In [ ]:
Panic=dict()
Panicx=["panicked","anxious","troubled","uncomfortable","stunned","mixed up",
"unsure","stuck","frozen","desperate"]
print("Panic Feelings Word Count")
for a in Panicx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Panic[a] = o
    except KeyError:
        continue
print(Panic)

In [ ]:
Anger=dict()
Angerx=["angered","annoyed","agitated","raging","furious","livid",
"bitter","Fed up","irritated","mad","critical","resentful","disgusted","outraged"]
print("Anger Feelings Word Count")
for a in Angerx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Anger[a] = o
    except KeyError:
        continue
print(Anger)

In [ ]:
Tired=dict()
Tiredx=["tired","indifferent","bored","drained","exhausted","dull","weary",
"powerless","burned out","fatigued","stale"]
print("Tiredness Feelings Word Count")
for a in Tiredx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Tired[a] = o
    except KeyError:
        continue
print(Tired)

In [ ]:
Confused=dict()
Confusedx=["shy","lost","unsure","uncertain","upset","doubtful","uncertain","indecisive",
"perplexed","confused","hesitant"]
print("Confused Feelings Word Count")
for a in Confusedx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Confused[a] = o
    except KeyError:
        continue
print(Confused)

In [ ]:
Embarassed=dict()
Embarassedx=["isolated","self-conscious","lonely","inferior","ashamed",
"pathetic","confused","guilty","repugnant","embarassed"]
print("Embarassed Feelings Word Count")
for a in Embarassedx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Embarassed[a] = o
    except KeyError:
        continue
print(Embarassed)

In [ ]:
Helped=dict()
Helpedx=["cherished","befriended","appreciated","understood","commended","empowered","accepted",
"blessed","healed","loved","saved","helped"]
print("Helped Feelings Word Count")
for a in Helpedx:
    try:
        o = words.loc[a]
        print(f"{str(o)} words {a}")
        Helped[a] = o
    except KeyError:
        continue
print(Helped)

In [ ]:
def graph_count_feeling(name):
    x=list()
    y=list()
    county= 0
    
    for key, value in name.items():
        x.append(key)
        y.append(value)
        county =  county + value
# _________________________________________________
    p = figure(x_range=x, plot_height=500, title="Types of Feelings vs Quantity")
    p.xaxis.axis_label = "Types of Feelings"
    p.yaxis.axis_label = 'Word Quantity'
    p.vbar(x=x, top=y, width=0.9)
    p.y_range.start = 0
    show(p)
    return county

In [ ]:
graph_count_feeling(Helped)

In [ ]:
graph_count_feeling(Embarassed)

In [ ]:
graph_count_feeling(Confused)

In [ ]:
graph_count_feeling(Tired)

In [ ]:
graph_count_feeling(Anger)

In [ ]:
graph_count_feeling(Panic)

In [ ]:
graph_count_feeling(Hurt)

In [ ]:
graph_count_feeling(Happy)

In [ ]:
graph_count_feeling(Sad)

**Total de Palavras Tristes:3374.
Total de Palavras Felizes:20867.
Total de Palavras de Ferir/Machucar:962.
Total de Palavras de Pânico:2417.
Total de Palavras de Raiva:421.
Total de Palavras de Cansaço:2468.
Total de Palavras de Confuso:3356.
Total de Palavras de Embaraço:638.
Total de Palavras de Ajudado:77319.**

No total, são 111822 palavras. Sendo:

3,0% Total de Palavras Tristes
18,7% Total de Palavras Felizes
0,9% Total de Palavras de Ferir/Machucar
2,2% Total de Palavras de Pânico
0,4% Total de Palavras de Raiva:
2,2% Total de Palavras de Cansaço
3,0% Total de Palavras de Confuso
0,6% Total de Palavras de Embaraço
69,1% Total de Palavras de Ajudado


***O estrondoso peso de palavras que expressam sensação de apoio, ajuda , atenção (70%) demonstra como os romanos colaboram e dão suporte aos visitantes.***